In [1]:
import numpy as np
import jax.numpy as jnp
import pytest
import ticktack
from ticktack import fitting

In [2]:
cbm = ticktack.load_presaved_model('Guttler15', production_rate_units='atoms/cm^2/s')
sf = fitting.SingleFitter(cbm, 'Guttler15', hemisphere="north")
sf.time_data = jnp.arange(200, 210)
sf.d14c_data_error = jnp.ones((sf.time_data.size,))
sf.d14c_data = jnp.array([-169.81482498, -168.05109886, -163.81278239, -158.13313339,
                          -153.23525037, -150.89762995, -151.11804461, -152.1036496 ,
                          -151.60492619, -151.60492619])
sf.start = np.nanmin(sf.time_data)
sf.end = np.nanmax(sf.time_data)
sf.burn_in_time = jnp.arange(sf.start - 1000, sf.start, 1.)
sf.oversample = 1008
sf.time_data_fine = jnp.linspace(sf.start, sf.end + 2, int(sf.oversample * (sf.end - sf.start + 2)))
sf.burnin_oversample = 1
sf.offset = 0
sf.annual = jnp.arange(sf.start, sf.end + 1)
sf.mask = jnp.in1d(sf.annual, sf.time_data)
sf.growth = jnp.array([0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0])

INFO[2022-02-14 16:47:05,287]: Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: 
INFO[2022-02-14 16:47:05,288]: Unable to initialize backend 'gpu': Not found: Could not find registered platform with name: "cuda". Available platform names are: Interpreter Host
INFO[2022-02-14 16:47:05,289]: Unable to initialize backend 'tpu': Invalid argument: TpuPlatform is not available.
WARNING[2022-02-14 16:47:05,289]: No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
mf = fitting.MultiFitter()
sf.compile_production_model(model="simple_sinusoid")
mf.add_SingleFitter(sf)
mf.add_SingleFitter(sf)
mf.compile()

In [4]:
out = mf.multi_likelihood(params=jnp.array([205., 1. / 12, jnp.pi / 2., 81. / 12]))
out

DeviceArray(-271736.26950342, dtype=float64)

In [5]:
out = sf.simple_sinusoid(200, jnp.array([205., 1./12, jnp.pi/2., 81./12]))
out

DeviceArray(2.1823329, dtype=float64)

In [6]:
out = sf.flexible_sinusoid(200, jnp.array([205., 1./12, jnp.pi/2., 81./12, 0.1]))
out

DeviceArray(2.04813439, dtype=float64)

In [7]:
sf.compile_production_model(model="simple_sinusoid")
a = sf.dc14(params=jnp.array([205., 1. / 12, jnp.pi / 2., 81./12]))
sf.compile_production_model(model="flexible_sinusoid")
b = sf.dc14(params=(jnp.array([205., 1./12, jnp.pi/2., 81./12, 0.1])))
sf.compile_production_model(model="control_points")
c = sf.dc14(params=jnp.ones(sf.control_points_time.size))
a, b, c

(DeviceArray([ 0.04804975,  0.63910482,  0.76016559,  0.37527574,
              -0.3795604 , 12.34915479, 14.50537971, 14.54228208,
              14.06061059, 13.67800247], dtype=float64),
 DeviceArray([ 0.02669429,  0.35505823,  0.42231421,  0.20848652,
              -0.21086689, 12.90361593, 15.365865  , 15.52502989,
              14.93705643, 14.24820176], dtype=float64),
 DeviceArray([-126.51396123, -126.55161057, -126.60292691, -126.65378533,
              -126.70581364, -126.75734358, -126.81007871, -126.85676223,
              -126.93691418, -127.52508047], dtype=float64))

dc14 fine

In [8]:
sf.compile_production_model(model="simple_sinusoid")
a = sf.dc14_fine(params=jnp.array([205., 1. / 12, jnp.pi / 2., 81./12]))[-9:]
sf.compile_production_model(model="flexible_sinusoid")
b = sf.dc14_fine(params=jnp.array([205., 1./12, jnp.pi/2., 81./12, 0.1]))[-9:]
sf.compile_production_model(model="control_points")
c = sf.dc14_fine(params=jnp.ones(sf.control_points_time.size))[-9:]
a, b, c

(DeviceArray([13.42488917, 13.42474906, 13.42460886, 13.42446859,
              13.42432823, 13.42418779, 13.42404727, 13.42390667,
              13.42376598], dtype=float64),
 DeviceArray([13.37697866, 13.37644154, 13.37590442, 13.37536731,
              13.37483019, 13.37429309, 13.37375598, 13.37321888,
              13.37268179], dtype=float64),
 DeviceArray([-129.65468629, -129.65650745, -129.65832902, -129.660151  ,
              -129.6619734 , -129.6637962 , -129.66561942, -129.66744304,
              -129.66926708], dtype=float64))

In [9]:
sf.compile_production_model(model="simple_sinusoid")
out = sf.log_likelihood(jnp.array([205., 1. / 12, jnp.pi / 2., 81./12]))
out

DeviceArray(-134749.41424852, dtype=float64)

In [10]:
sf.compile_production_model(model="control_points")
out = sf.log_likelihood_gp(jnp.ones(sf.control_points_time.size))
out

DeviceArray(-10.21255349, dtype=float64)

In [11]:
sf.compile_production_model(model="control_points")
out = sf.log_joint_likelihood_gp(jnp.ones(sf.control_points_time.size), jnp.zeros((sf.control_points_time.size)),
                                                        jnp.ones(sf.control_points_time.size) * 100)
out

DeviceArray(-4848.15844474, dtype=float64)

In [12]:
out = sf.grad_neg_log_joint_likelihood_gp(jnp.ones(sf.control_points_time.size))
out

DeviceArray([40903.92842731,   603.14554327,   455.32578778,
               400.57669555,   335.71502475,   280.95206461,
               233.92081014,   148.30657001,   162.21014777],            dtype=float64)